In [1]:
import warnings
warnings.simplefilter('ignore')
from utils import *
import pickle
import pandas as pd
from collections import defaultdict
from scipy.stats import pearsonr

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import ensemble

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit import DataStructs

import matplotlib.pyplot as plt
import seaborn as sns
from statannotations.Annotator import Annotator

In [2]:
sns.set_style('ticks', 
              {'font.sans-serif':['SimHei', 'Arial'],
                'text.color': 'black',
                'xtick.color': 'black',
                'ytick.color': 'black',
                })
# plt.rcParams.update({'font.size':20})
colors = sns.color_palette('Paired')

In [3]:
def get_one_hot_cid(cid_set, cid_list):

    label_encoder = LabelEncoder()
    onehot_encoder = OneHotEncoder(sparse=False)

    label_encoder_fit = label_encoder.fit(cid_set)
    integer_encoded = label_encoder_fit.transform(cid_set)
    onehot_encoder_fit = onehot_encoder.fit(integer_encoded.reshape(-1, 1))
    cid_integer_encoded = label_encoder_fit.transform(cid_list)
    one_hot_encoded_cid = onehot_encoder_fit.transform(cid_integer_encoded.reshape(-1, 1))

    return one_hot_encoded_cid, onehot_encoder_fit, label_encoder_fit

In [4]:
cell_ls = ['A375', 'HA1E', 'HELA', 'HT29', 'MCF7', 'PC3', 'YAPC']
with open('../data/LINCS2020/KPGT_emb2304.pickle', 'rb') as f:
    smi2emb = pickle.load(f)
    
with open('../data/LINCS2020/idx2smi.pickle', 'rb') as f:
    idx2smi = pickle.load(f)
with open('../data/LINCS2020/smi2idx.pickle', 'rb') as f:
    smi2idx = pickle.load(f)
print(len(smi2emb.keys()))

8316


### TranSiGen ΔX' (test_external)

In [5]:
pred_dir = '../results/baseline/trained_models_7_cell_smiles_split/895834/feature_KPGT_init_pretrain_shRNA/predict/'
data_LINCS = load_from_HDF(pred_dir + 'test_external_prediction_profile.h5')
smi_ls = []    
for idx in range(data_LINCS['cid'].shape[0]):
    smi = idx2smi[data_LINCS['cp_id'][idx]]
    smi_ls.append(smi)
data_LINCS['canonical_smiles'] = np.array(smi_ls)

for k in data_LINCS.keys():
    print(k, data_LINCS[k].shape)

cid (25126,)
cp_id (25126,)
sig (25126,)
x1 (25126, 978)
x2 (25126, 978)
x2_pred (25126, 978)
x2_rec (25126, 978)
canonical_smiles (25126,)


In [6]:
ECFP_array = []
for smi in data_LINCS['canonical_smiles']:
    mol = Chem.MolFromSmiles(smi)
    ECFP = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    ECFP_array.append(ECFP)
data_LINCS['ECFP4'] = np.array(ECFP_array)

KPGT_array = []
for smi in data_LINCS['canonical_smiles']:
    KPGT_array.append(smi2emb[smi])
data_LINCS['KPGT'] = np.array(KPGT_array)

x1_array = data_LINCS['x1']
DEG_array = data_LINCS['x2'] - data_LINCS['x1']
DEG_rec_array = data_LINCS['x2_rec'] - data_LINCS['x1']
DEG_pred_array = data_LINCS['x2_pred'] - data_LINCS['x1']
ECFP_array = data_LINCS['ECFP4']
KPGT_array = data_LINCS['KPGT']

ECFP_cid_x1_array = np.concatenate((ECFP_array, x1_array), axis=1)
print(ECFP_cid_x1_array.shape)
KPGT_cid_x1_array = np.concatenate((KPGT_array, x1_array), axis=1)
print(KPGT_cid_x1_array.shape)

(25126, 3026)
(25126, 3282)


In [7]:
## information for all cell line, molecules
df_LINCS = pd.DataFrame(data_LINCS['cid'], columns=['cid'])
df_LINCS['canonical_smiles'] = data_LINCS['canonical_smiles']
df_LINCS

,cid,canonical_smiles
0,PC3,CN(C)C(=O)[C@H]1[C@H](CO)[C@H]2Cn3c(ccc(C4=CCC...
1,PC3,COc1cc(ccc1Nc2ncc3N(C)C(=O)CCN(C4CCC(O)CC4)c3n...
2,MCF7,COc1ccc2C(=O)/C(=Cc3ccc(OC)c(OC)c3)/COc2c1
3,MCF7,OC(=O)CCC(NS(=O)(=O)c1ccc(cc1)c2ccccc2)C(=O)O
4,MCF7,Oc1ccc(-c2nn[nH]n2)c2cccnc12
...,...,...
25121,PC3,C[C@H](CO)N1C[C@@H](C)[C@@H](CN(C)S(=O)(=O)c2c...
25122,MCF7,COc1ccc(cc1)C(=O)NC(c2ccc(OC)c(OC)c2)c3cc(Cl)c...
25123,PC3,CCN(C)C(=O)Oc1cccc(c1)[C@H](C)N(C)C
25124,A375,CC[C@@H]1C=C(C)C[C@H](C)C[C@H](OC)[C@@H]2O[C@]...


### DLEPS

In [8]:
df_DLEPS_external = pd.read_csv('../results/baseline/DLEPS/smiles_split/result_test_external_seed343.csv')
DLEPS_pred_array = np.array(df_DLEPS_external.iloc[:, 1:])
print(DLEPS_pred_array.shape, type(DLEPS_pred_array))
DLEPS_pred_array

(25126, 978) <class 'numpy.ndarray'>


array([[-0.49215972,  0.54963338,  0.29133952, ..., -0.84925705,
        -0.00142619,  0.4664081 ],
       [-0.84851676,  0.55724311,  0.34684628, ..., -1.13050663,
         0.01257842,  0.44271934],
       [ 1.24347758,  0.93602711, -0.12212732, ...,  1.16335428,
        -0.12323196,  0.72062916],
       ...,
       [-0.11247773,  0.64319599,  0.21654259, ..., -0.27496064,
        -0.04910432,  0.52765197],
       [-0.75984836,  0.33571479,  0.32759017, ..., -1.07286894,
         0.01936504,  0.28974941],
       [ 0.434286  ,  0.51672924, -0.01642013, ...,  0.38941333,
        -0.04644619,  0.39385477]])

In [9]:
cell_ls = ['A375', 'HA1E', 'HELA', 'HT29', 'MCF7', 'PC3', 'YAPC']
one_hot_encoded_cid, _, _ = get_one_hot_cid(cell_ls, list(data_LINCS['cid']))
one_hot_encoded_cid_DLEPS = one_hot_encoded_cid.copy()
for idx in range(DLEPS_pred_array.shape[0]):
    if DLEPS_pred_array[idx,:].any() == 0:
        one_hot_encoded_cid_DLEPS[idx,:] = np.array([0]*7)
DLEPS_pred_cid_array = np.concatenate((DLEPS_pred_array, one_hot_encoded_cid_DLEPS), axis=1)
DLEPS_pred_cid_array.shape

(25126, 985)

### DeepCE

In [10]:
df_DeepCE_external = pd.read_csv('../results/baseline/DeepCE/smiles_split/result_test_external_seed343.csv')
DeepCE_pred_array = np.array(df_DeepCE_external.iloc[:, 5:])
print(DeepCE_pred_array.shape, type(DeepCE_pred_array))
DeepCE_pred_array

(25126, 978) <class 'numpy.ndarray'>


array([[-0.97417456,  0.94665819, -0.09210257, ..., -0.98393347,
        -0.0360845 ,  0.32205338],
       [ 0.82055187,  0.93450588,  0.58601191, ..., -0.347447  ,
         0.1559524 ,  1.15640839],
       [ 0.94290865,  1.22360912, -0.06593236, ...,  0.29021719,
         0.01664902,  1.11445657],
       ...,
       [ 0.93245108,  0.92178554,  0.21250249, ...,  2.34908056,
        -0.33434472,  0.53529965],
       [-1.32865398, -0.10063478,  0.14201809, ..., -1.04868236,
        -0.20480477, -0.23018641],
       [ 1.06971797,  1.37444122, -0.40474831, ...,  2.15661409,
        -0.39273902,  0.52712886]])

### CIGER

In [11]:
df_CIGER_external = pd.read_csv('../results/baseline/CIGER/smiles_split/result_test_external_seed343_real.csv')
CIGER_pred_array = np.array(df_CIGER_external.iloc[:, 5:])
print(CIGER_pred_array.shape, type(CIGER_pred_array))
CIGER_pred_array

(25126, 978) <class 'numpy.ndarray'>


array([[-2.19572544,  1.5308708 ,  0.03873972, ...,  0.38758346,
         0.76600462, -1.19404328],
       [ 5.49974155,  3.44760108,  1.09678066, ...,  6.2509346 ,
         0.36355799,  2.71384668],
       [ 1.54190314,  2.68659329,  0.48239705, ...,  0.05228177,
         0.44299695,  2.21594024],
       ...,
       [ 3.55539727,  1.89573467,  0.69547939, ...,  5.47067928,
         0.85276121,  3.90251541],
       [-0.86126018,  1.45067167, -0.09999803, ...,  0.05743099,
        -0.59741366,  0.84147489],
       [ 2.24885273,  5.6389184 , -1.67175472, ...,  6.15553093,
         1.96859515,  1.8116442 ]])

## train model for drug response prediction

In [117]:
df_grid_search_results = pd.read_csv('../results/5.Drug_response_prediction/grid_search_param.csv')
df_data = pd.read_csv('../results/5.Drug_response_prediction/data.csv')

index_train = []
for _, row in df_data[df_data['split']=='train'].iterrows():
    index_train.append( df_LINCS[(df_LINCS['canonical_smiles']==row['canonical_smiles']) & (df_LINCS['cid']==row['cid'])].index[0] )
index_test = []
for _, row in df_data[df_data['split']=='test'].iterrows():
    index_test.append( df_LINCS[(df_LINCS['canonical_smiles']==row['canonical_smiles']) & (df_LINCS['cid']==row['cid'])].index[0] )

df_metrics = pd.DataFrame(columns=['Pearson', 'RMSE', 'R2', 'data'])
predict_result = defaultdict(list)
random_seed_ls  = [0, 1000, 10000, 100000, 1000000]
for data_name, data_sub in zip(['ECFP4+$\mathit{X}_1$', 'KPGT+$\mathit{X}_1$', '$\mathit{\Delta}X$', 'TranSiGen', 'DLEPS', 'DeepCE', 'CIGER'], 
                               [ECFP_cid_x1_array, KPGT_cid_x1_array, DEG_array, DEG_pred_array, DLEPS_pred_cid_array, DeepCE_pred_array, CIGER_pred_array]):
    print(data_name)
    best_params = df_grid_search_results[(df_grid_search_results['data']== data_name)].reset_index(drop=True).to_dict(orient='index')[0]
    print(best_params)
    metrics_dict = defaultdict(list)
    run_idx = 0
    for run_random_seed in random_seed_ls:
        clf = ensemble.RandomForestRegressor(max_depth=best_params['max_depth'], 
                                             n_estimators=best_params['n_estimators'], 
                                             max_features='auto', 
                                             criterion=best_params['criterion'], 
                                             oob_score = best_params['oob_score'],
                                             random_state=run_random_seed)

            
        data_train, data_test = data_sub[index_train], data_sub[index_test]
        label_train, label_test = np.array(df_data[df_data['split']=='train']['area_under_curve']), np.array(df_data[df_data['split']=='test']['area_under_curve'])
        smiles_train, smiles_test = np.array(df_data[df_data['split']=='train']['canonical_smiles']), np.array(df_data[df_data['split']=='test']['canonical_smiles'])
        smi_idx_train, smi_idx_test = np.array([smi2idx[smi] for smi in smiles_train]), np.array([smi2idx[smi] for smi in smiles_test])
        cid_train, cid_test = np.array(df_data[df_data['split']=='train']['cid']), np.array(df_data[df_data['split']=='test']['cid'])

        if data_name == 'DLEPS':
            train_index_new = np.where(data_train.any(axis=1))[0]
            print('train_index_new:', len(train_index_new))
            data_train = data_train[train_index_new,:]
            label_train = label_train[train_index_new]
            smi_idx_train = smi_idx_train[train_index_new]
            cid_train = cid_train[train_index_new]
            smi_idx_train_DLEPS = smi_idx_train
            
            test_index_new = np.where(data_test.any(axis=1))[0]
            print('test_index_new:', len(test_index_new))
            data_test = data_test[test_index_new,:]
            label_test= label_test[test_index_new]
            smi_idx_test = smi_idx_test[test_index_new]
            cid_test = cid_test[test_index_new]
            smi_idx_test_DLEPS = smi_idx_test
        
        print('TRAIN:', data_train.shape, 'TEST:', data_test.shape)
        print('train_smiles:', len(set(smi_idx_train)), 'test_smiles:', len(set(smi_idx_test)), 'common:', set(smi_idx_train) & set(smi_idx_test))
        print('train_cid:', len(set(cid_train)), 'test_cid:', len(set(cid_test)), 'common:', set(cid_train) & set(cid_test))

        clf = clf.fit(data_train, label_train)
        data_test_pred = clf.predict(data_test)
        if data_name == 'TranSiGen':
            predict_result['cp_id'] += [list(smi_idx_test)]
            predict_result['cid'] += [list(cid_test)]
            predict_result['label'] += [list(label_test)]
            predict_result['idx'] += [[run_idx] *len(list(label_test))]
        elif data_name == 'DLEPS':
            predict_result['DLEPS_cp_id'] += [list(smi_idx_test)]
            predict_result['DLEPS_cid'] += [list(cid_test)]
            predict_result['DLEPS_label'] += [list(label_test)]
            predict_result['DLEPS_idx'] += [[run_idx] *len(list(label_test))]
        
        predict_result[data_name] += [list(data_test_pred)]
        metrics_dict['Pearson'] += [pearsonr(label_test, data_test_pred)[0] ]
        metrics_dict['RMSE'] += [np.sqrt(mean_squared_error(label_test, data_test_pred))]
        metrics_dict['R2'] += [r2_score(label_test, data_test_pred)]
        metrics_dict['random_seed'] += [run_random_seed]

        df = pd.DataFrame(cid_test, columns=['cid'])
        df['cp_id'] = smi_idx_test
        df['AUC'] = label_test
        df['AUC_pred'] = data_test_pred
        print(df.shape)
        run_idx +=1
    df_metrics_tmp = pd.DataFrame.from_dict(metrics_dict)
    df_metrics_tmp.loc[:, 'data'] = data_name
    df_metrics = pd.concat([df_metrics, df_metrics_tmp])

with open('../results/5.Drug_response_prediction/prediction.pkl', 'wb') as f:
    pickle.dump(predict_result, f)
df_metrics.to_csv('../results/5.Drug_response_prediction/results.csv', index=False)

df_summary = df_metrics.groupby(['data']).agg(func=['mean', 'std'])
round(df_summary,3) 

ECFP4+$\mathit{X}_1$
{'criterion': 'mse', 'max_depth': 40, 'n_estimators': 250, 'oob_score': True, 'data': 'ECFP4+$\\mathit{X}_1$'}
TRAIN: (643, 3026) TEST: (160, 3026)
train_smiles: 214 test_smiles: 53 common: set()
train_cid: 4 test_cid: 4 common: {'HT29', 'MCF7', 'A375', 'PC3'}
(160, 4)
TRAIN: (643, 3026) TEST: (160, 3026)
train_smiles: 214 test_smiles: 53 common: set()
train_cid: 4 test_cid: 4 common: {'HT29', 'MCF7', 'A375', 'PC3'}
(160, 4)
TRAIN: (643, 3026) TEST: (160, 3026)
train_smiles: 214 test_smiles: 53 common: set()
train_cid: 4 test_cid: 4 common: {'HT29', 'MCF7', 'A375', 'PC3'}
(160, 4)
TRAIN: (643, 3026) TEST: (160, 3026)
train_smiles: 214 test_smiles: 53 common: set()
train_cid: 4 test_cid: 4 common: {'HT29', 'MCF7', 'A375', 'PC3'}
(160, 4)
TRAIN: (643, 3026) TEST: (160, 3026)
train_smiles: 214 test_smiles: 53 common: set()
train_cid: 4 test_cid: 4 common: {'HT29', 'MCF7', 'A375', 'PC3'}
(160, 4)
KPGT+$\mathit{X}_1$
{'criterion': 'mae', 'max_depth': 10, 'n_estimators':

Pearson          RMSE            R2        random_seed  \
                        mean    std   mean    std   mean    std        mean   
data                                                                          
$\mathit{\Delta}X$     0.521  0.026  3.367  0.030  0.152  0.015    222200.0   
CIGER                 -0.074  0.015  3.926  0.016 -0.153  0.010    222200.0   
DLEPS                  0.122  0.023  3.825  0.045 -0.094  0.026    222200.0   
DeepCE                -0.069  0.044  3.908  0.042 -0.142  0.025    222200.0   
ECFP4+$\mathit{X}_1$   0.507  0.013  3.163  0.025  0.252  0.012    222200.0   
KPGT+$\mathit{X}_1$    0.339  0.018  3.527  0.022  0.070  0.012    222200.0   
TranSiGen              0.574  0.016  3.094  0.022  0.284  0.010    222200.0   

                                  
                             std  
data                              
$\mathit{\Delta}X$    436817.124  
CIGER                 436817.124  
DLEPS                 436817.124  
DeepCE                436817.124  
ECFP4+$\mathit{X}_1$  436817.124  
KPGT+$\mathit{X}_1$   436817.124  
TranSiGen             436817.124

## ranking results

In [124]:
te_fold_nums = 2
df_sim_and_result = pd.DataFrame(predict_result['label'][te_fold_nums], columns=['AUC'])
df_sim_and_result['cp_id'] = predict_result['cp_id'][te_fold_nums]
df_sim_and_result['cid'] = predict_result['cid'][te_fold_nums]
label_ls = []
for idx, row in df_sim_and_result.iterrows():
    if row['AUC']< 5.5:
        label_ls.append(int(1))
    else:
        label_ls.append(int(0))
df_sim_and_result['label'] = label_ls
for data_name in ['$\mathit{\Delta}X$', 'TranSiGen', 'ECFP4+$\mathit{X}_1$', 'KPGT+$\mathit{X}_1$', 'DeepCE', 'CIGER']:
    df_sim_and_result[data_name] = predict_result[data_name][te_fold_nums]
df_sim_and_result

643 160
common: set()


,AUC,cp_id,cid,label,$\mathit{\Delta}X$,TranSiGen,ECFP4+$\mathit{X}_1$,KPGT+$\mathit{X}_1$,DeepCE,CIGER
0,12.31900,4807,MCF7,0,10.733256,11.724784,13.316633,11.817520,10.567892,12.115459
1,15.39800,2622,MCF7,0,10.933174,10.438983,11.081039,11.618882,10.754904,9.012416
2,10.47685,6155,PC3,0,12.007072,12.361170,12.036789,9.570461,9.328546,9.914995
3,14.46900,310,MCF7,0,11.552569,10.534148,11.690706,10.115892,10.029063,10.706510
4,14.46400,8037,HT29,0,9.782799,9.607356,8.119071,9.484736,10.610371,10.559199
...,...,...,...,...,...,...,...,...,...,...
155,16.32850,3599,A375,0,12.031787,12.483057,12.621768,12.322975,11.759227,10.676409
156,11.27410,5824,MCF7,0,11.223015,9.746259,9.390232,9.075557,10.410073,9.166461
157,11.25410,921,A375,0,10.874498,10.431560,9.742874,10.289987,11.420037,11.721807
158,12.61500,5627,PC3,0,10.535051,13.209640,12.893679,11.151481,10.559931,10.252259


In [129]:
df_sim_and_result_DLEPS = pd.DataFrame(predict_result['DLEPS_label'][te_fold_nums], columns=['AUC'])
df_sim_and_result_DLEPS['cp_id'] = predict_result['DLEPS_cp_id'][te_fold_nums]
df_sim_and_result_DLEPS['cid'] = predict_result['DLEPS_cid'][te_fold_nums]

label_ls = []
for idx, row in df_sim_and_result_DLEPS.iterrows():
    if row['AUC']< 5.5:
        label_ls.append(int(1))
    else:
        label_ls.append(int(0))
df_sim_and_result_DLEPS['label'] = label_ls
for data_name in ['DLEPS']:
    df_sim_and_result_DLEPS[data_name] = predict_result[data_name][te_fold_nums]
df_sim_and_result_DLEPS

,AUC,cp_id,cid,label,DLEPS
0,12.31900,4807,MCF7,0,10.944927
1,15.39800,2622,MCF7,0,13.327067
2,10.47685,6155,PC3,0,11.008116
3,14.46900,310,MCF7,0,9.833601
4,14.46400,8037,HT29,0,10.164303
...,...,...,...,...,...
155,16.32850,3599,A375,0,11.831274
156,11.27410,5824,MCF7,0,10.245170
157,11.25410,921,A375,0,8.451948
158,12.61500,5627,PC3,0,11.195564
